In [1]:
import requests

In [2]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
toronto_neighborhood_page = requests.get(wikipedia_link)

In [4]:
page = toronto_neighborhood_page.text

## Using HTMLParser library and adding custom functions

In [6]:
from html.parser import HTMLParser
from html.entities import name2codepoint


class MyHTMLParser(HTMLParser):
    _data = []
    _attr = ('xxx', 'zzzzz')
    _loc_url = []
    _postal = []
    _borough = []
    _neighborhood = []
    _na = []
    _b = 0
    _n = 0
    _l = 'e'
    _le = 'n' 
    m_dup = 0
    _open = 0
    
    def handle_starttag(self, tag, attrs):
        #print( "Start tag:", tag)
        #self._le = self._l
        for attr in attrs:
            if attr[1].find('/wiki/') != -1:
                #print('ATTR[1] WIKI DETECTED')
                self._attr = attr
            #print("     attr:", attr, ' type :', type(attr))

    def handle_endtag(self, tag):
        self._attr = ('xxx', 'zzzzz')
        #print ("End tag  :", tag)

    def handle_data(self, data):
        #self._data.append(data)
        #print('B : ', self._b, ' DATAM b4 :', data )
       
        if data.find('M') != -1 and len(data) == 3:
           #print('DATAM DETECTED')
            self._le = data
            #print(' SLEF._LE', self._le)
            if len(self._postal) !=0:
                if data != self._postal[len(self._postal) - 1]:                
                    self.m_dup = 0     
                    #print('BOROUGH COMPARE B4 : ', self._borough[len(self._borough) - 1], ' DATA: ', data)
                else:
                    self.m_dup = 1
                    self._b = 1
                    #print('BOROUGH COMPARE : ', self._borough[len(self._borough) - 1], ' DATA: ', data)
                    if self._borough[len(self._borough) - 1] != data:
                        self._b = 1
                        
                
            
        if self._b == 1 and data.find('Not assigned') != -1:
            self._neighborhood.append(self._borough[len(self._borough) - 1])
            self._b = 0
        if self._b == 1 and data.find('Not assigned') == -1 and len(data) > 3 and self._attr[1].find('zzzz') != -1:
            #print('UNLINKED DATA DETECTED', data, ' test self._attr[1]: ', self._attr[1])
            if self.m_dup != 0:
                self._neighborhood[len(self._neighborhood) - 1] = self._neighborhood[len(self._neighborhood) - 1] + "," + data
                self._b = 0
            else:  
                self._neighborhood.append(data)
                self._b = 0                    
                
        if data == 'North York':
            self._open = 1
            
        if self._attr[1].find('/wiki/') != -1 and self._open==1:
            #print(' M B4 LOGIC : ', self.m_dup)
            if self._b == 0 and self.m_dup == 0:
                #print('B = 0 DETECTED', " M = 0", ' le :', self._le)
                self._postal.append(self._le)
                self._borough.append(data)
                self._b = 1    
            elif self._b == 1 and self.m_dup == 0:
                #print('B = 1 DETECTED')
                self._neighborhood.append(data)
                self._b = 0
            elif self._b == 1 and self.m_dup == 1:
                if data != self._borough[len(self._borough) - 1]:
                    #print('DUPLICATE IDENTIFIED NEIGH: ', data,  ' b or n: ', self._borough[len(self._borough) - 1] )
                    self._neighborhood[len(self._neighborhood) - 1] = self._neighborhood[len(self._neighborhood) - 1] + "," + data
                    self._b = 0
                    
                
            
        if data == 'South of Bloor':
            self._open = 0   

parser = MyHTMLParser()
parser.feed(page)

### Populate DataFramae using the data optained from the parser

In [7]:
import pandas as pd
t_columns = ['PostalCode','Borough','Neighborhood']
n_df = pd.DataFrame(columns=t_columns)
n_df[t_columns[0]] = parser._postal
n_df[t_columns[1]] = parser._borough
n_df[t_columns[2]] = parser._neighborhood
n_df.head(101)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North\n
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District\n"


In [8]:
n_df.shape

(101, 3)

In [182]:
### Trying to populate the location using geocoders library. It produces inconsistent results

In [184]:
address = 'Etobicoke  , Toronto'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
lat = location.latitude
lng = location.longitude
print(i, 'Borough : ',' - ',lat, ' : ', lng)

100 Borough :   -  43.6435559  :  -79.5656326


In [185]:
def find_location():
    lat = []
    lng = []
    t_borough = []
    ex_borough = []
    t_pc = []
    ex_pc = []
    t_neighborhood = []
    
    for i in range(0, len(parser._borough)):
        try:
            address = '{},Toronto'.format(parser._borough[i])
            geolocator = Nominatim(user_agent="toronto_explorer")
            location = geolocator.geocode(address)
            lat.append(location.latitude)
            lng.append(location.longitude)
            t_borough.append(parser._borough[i])
            t_pc.append(parser._postal[i])
            t_neighborhood.append(parser._neighborhood[i])
            #print(i,'The geograpical coordinate of {} are {}, {}.'.format(parser._borough[i],location.latitude, location.longitude))
        
        except:
            ex_borough.append(parser._borough[i])
            ex_pc.append(parser._postal[i])
            #print(i,' EXCEPTION CAUGHT : ', parser._borough[i])
        
    return lat, lng, t_borough,t_neighborhood, t_pc, ex_borough, ex_pc

In [186]:
t_lat, t_lng, t_borough,t_neighborhood,t_pc, ex_borough, ex_pc = find_location()

In [187]:
t_columns = ['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']
toronto_data = pd.DataFrame(columns= t_columns)
toronto_data[t_columns[0]] = t_pc
toronto_data[t_columns[1]] = t_borough
toronto_data[t_columns[2]] = t_neighborhood
toronto_data[t_columns[3]] = t_lat
toronto_data[t_columns[4]] = t_lng

toronto_data.head(98)           


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4A,North York,Victoria Village,43.770817,-79.413300
1,M6A,North York,"Lawrence Heights,Lawrence Manor",43.770817,-79.413300
2,M7A,Queen's Park,Queen's Park,43.659980,-79.390369
3,M9A,Etobicoke,Islington Avenue,43.643556,-79.565633
4,M1B,Scarborough,"Rouge,Malvern",43.773077,-79.257774
5,M3B,North York,Don Mills North\n,43.770817,-79.413300
6,M4B,East York,"Woodbine Gardens,Parkview Hill",43.691339,-79.327821
7,M5B,Downtown Toronto,"Ryerson,Garden District\n",43.654174,-79.380812
8,M6B,North York,Glencairn,43.770817,-79.413300
9,M9B,Etobicoke,"Cloverdale\n,Islington,Martin Grove\n,Princess...",43.643556,-79.565633


### Importing location data from local csv file.


In [52]:
toronto_data_local = pd.read_csv('Dataset/Geospatial_Coordinates.csv')
toronto_data_local.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [56]:
pc_local = toronto_data_local['Postal Code'].values
lat_local = toronto_data_local['Latitude'].values
lng_local = toronto_data_local['Longitude'].values

In [57]:
toronto_data_dict = {}
for i in range(0, len(pc_local)):
    toronto_data_dict[pc_local[i]] = [lat_local[i], lng_local[i]]

               
for ind, key in enumerate(toronto_data_dict):
    print(ind, ' : ',key,' :', toronto_data_dict[key])

0  :  M1B  : [43.806686299999996, -79.19435340000001]
1  :  M1C  : [43.7845351, -79.16049709999999]
2  :  M1E  : [43.7635726, -79.1887115]
3  :  M1G  : [43.7709921, -79.21691740000001]
4  :  M1H  : [43.773136, -79.23947609999999]
5  :  M1J  : [43.7447342, -79.23947609999999]
6  :  M1K  : [43.7279292, -79.26202940000002]
7  :  M1L  : [43.711111700000004, -79.2845772]
8  :  M1M  : [43.716316, -79.23947609999999]
9  :  M1N  : [43.692657000000004, -79.2648481]
10  :  M1P  : [43.7574096, -79.27330400000001]
11  :  M1R  : [43.750071500000004, -79.2958491]
12  :  M1S  : [43.7942003, -79.26202940000002]
13  :  M1T  : [43.7816375, -79.3043021]
14  :  M1V  : [43.8152522, -79.2845772]
15  :  M1W  : [43.799525200000005, -79.3183887]
16  :  M1X  : [43.836124700000006, -79.20563609999999]
17  :  M2H  : [43.8037622, -79.3634517]
18  :  M2J  : [43.7785175, -79.3465557]
19  :  M2K  : [43.7869473, -79.385975]
20  :  M2L  : [43.7574902, -79.37471409999999]
21  :  M2M  : [43.789053, -79.40849279999999]
22

### Populating the existing postal data with the imported file.

In [59]:
def final_data():
    final_lat = []
    final_lng = []
    final_pc = []
    final_bor = []
    final_nei = []

    for ind, key in enumerate(parser._postal):
        final_lat.append(toronto_data_dict[key][0])
        final_lng.append(toronto_data_dict[key][1])
        final_pc.append(parser._postal[ind])
        final_bor.append(parser._borough[ind])
        final_nei.append(parser._neighborhood[ind])
        
    return final_pc, final_bor, final_nei, final_lat, final_lng

In [60]:
final_pc, final_bor, final_nei, final_lat, final_lng = final_data()

In [62]:
f_toronto_data = pd.DataFrame(columns = t_columns)
f_toronto_data[t_columns[0]] = final_pc
f_toronto_data[t_columns[1]] = final_bor
f_toronto_data[t_columns[2]] = final_nei
f_toronto_data[t_columns[3]] = final_lat
f_toronto_data[t_columns[4]] = final_lng

In [63]:
f_toronto_data.shape

(101, 5)

In [64]:
f_toronto_data.head(101)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North\n,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District\n",43.657162,-79.378937


In [36]:
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
t_latitude = location.latitude
t_longitude = location.longitude
print(t_latitude, t_longitude) 

43.653963 -79.387207


In [65]:
import folium

map_toronto = folium.Map(location=[t_latitude, t_longitude], zoom_start=11)

for lat, lng, label in zip(f_toronto_data['Latitude'], f_toronto_data['Longitude'], f_toronto_data['Borough']):
    label=folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.5,
        parse_html=False
    ).add_to(map_toronto)
    
map_toronto  

In [66]:
CLIENT_ID = 'GHLTJRBOBDN41UDNHT42G4XDZBVNHT1H3HFHTNHDEZ1ESYHK'
CLIENT_SECRET = 'PL0PZAVLOYCQMDXOYCL1YAGWKLUTGJJP25QEFDNYXVO15ZBA'
VERSION = '20180604'

In [67]:
LIMIT = 100
RADIUS = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, CLIENT_SECRET, VERSION, t_latitude, t_longitude, RADIUS, LIMIT
    )
    
url  

'https://api.foursquare.com/v2/venues/explore?&client_id=GHLTJRBOBDN41UDNHT42G4XDZBVNHT1H3HFHTNHDEZ1ESYHK&client_secret=PL0PZAVLOYCQMDXOYCL1YAGWKLUTGJJP25QEFDNYXVO15ZBA&v=20180604&ll=43.653963,-79.387207&radius=500&limit=100'

### Importing data from Foursquare API

In [68]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c6a5e7c6a60714c7fad25cc'},
 'response': {'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 70,
  'suggestedBounds': {'ne': {'lat': 43.6584630045, 'lng': -79.38099903084075},
   'sw': {'lat': 43.649462995499995, 'lng': -79.39341496915925}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5227bb01498e17bf485e6202',
       'name': 'Downtown Toronto',
       'location': {'lat': 43.65323167517444,
        'lng': -79.38529600606677,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65323167517444,
          'lng': -79.38529600606677}],
        'distance': 174,
        'cc': 'CA',
        'city': 'Toronto',
     

In [69]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Venues

In [71]:
from pandas.io.json import json_normalize


venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns


venues = results['response']['groups'][0]['items']

venues

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '5227bb01498e17bf485e6202',
   'name': 'Downtown Toronto',
   'location': {'lat': 43.65323167517444,
    'lng': -79.38529600606677,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.65323167517444,
      'lng': -79.38529600606677}],
    'distance': 174,
    'cc': 'CA',
    'city': 'Toronto',
    'state': 'ON',
    'country': 'Canada',
    'formattedAddress': ['Toronto ON', 'Canada']},
   'categories': [{'id': '4f2a25ac4b909258e854f55f',
     'name': 'Neighborhood',
     'pluralName': 'Neighborhoods',
     'shortName': 'Neighborhood',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/neighborhood_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'groups': []}},
  'referralId': 'e-0-5227bb01498e17bf485e6202-0'},
 {'reasons': {'count': 0,
   'items': [{'summar

In [72]:
nearby_venues = json_normalize(venues)
nearby_venues.head(3)

,reasons.count,reasons.items,referralId,venue.categories,venue.id,venue.location.address,venue.location.cc,venue.location.city,venue.location.country,venue.location.crossStreet,...,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.neighborhood,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups,venue.venuePage.id
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-5227bb01498e17bf485e6202-0,"[{'id': '4f2a25ac4b909258e854f55f', 'name': 'N...",5227bb01498e17bf485e6202,NaN,CA,Toronto,Canada,NaN,...,"[{'label': 'display', 'lat': 43.65323167517444...",43.653232,-79.385296,NaN,NaN,ON,Downtown Toronto,0,[],NaN
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4ad4c05ef964a520e2f620e3-1,"[{'id': '4bf58dd8d48988d18f941735', 'name': 'A...",4ad4c05ef964a520e2f620e3,55 Centre Avenue,CA,Toronto,Canada,University Ave. and Dundas St W.,...,"[{'label': 'display', 'lat': 43.65439630500274...",43.654396,-79.386500,NaN,M5G 2H5,ON,Textile Museum of Canada,0,[],56305293
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-504bbf2ce4b0168121235cbe-2,"[{'id': '55a59bace4b013909087cb24', 'name': 'R...",504bbf2ce4b0168121235cbe,179 Dundas St. W,CA,Toronto,Canada,btwn Centre Ave. & Chestnut St.,...,"[{'label': 'display', 'lat': 43.65515746756124...",43.655157,-79.386501,NaN,M5G 1Z8,ON,Sansotei Ramen 三草亭,0,[],NaN


In [73]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]
nearby_venues.head(2)

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Downtown Toronto,"[{'id': '4f2a25ac4b909258e854f55f', 'name': 'N...",43.653232,-79.385296
1,Textile Museum of Canada,"[{'id': '4bf58dd8d48988d18f941735', 'name': 'A...",43.654396,-79.386500


In [74]:
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.head(3)

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Textile Museum of Canada,Art Museum,43.654396,-79.386500
2,Sansotei Ramen 三草亭,Ramen Restaurant,43.655157,-79.386501


In [75]:
print('{} venues were returned by four square.'.format(nearby_venues.shape[0]))

70 venues were returned by four square.


In [76]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Entire venues in Toronto

In [89]:
toronto_venues = getNearbyVenues(names=f_toronto_data['Neighborhood'],
                                   latitudes=f_toronto_data['Latitude'],
                                   longitudes=f_toronto_data['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront,Regent Park
Lawrence Heights,Lawrence Manor
Queen's Park
Islington Avenue
Rouge,Malvern
Don Mills North

Woodbine Gardens,Parkview Hill
Ryerson,Garden District

Glencairn
Cloverdale
,Islington,Martin Grove
,Princess Gardens,West Deane Park
Highland Creek,Rouge Hill,Port Union
Flemingdon Park,Don Mills South

Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens
,Eringate
,Markland Wood,Old Burnhamthorpe

Guildwood
,Morningside,West Hill
The Beaches
Berczy Park
Woburn
Leaside
Central Bay Street

Christie

Cedarbrae
Hillcrest Village
Bathurst Manor,Downsview North
,Wilson Heights
Thorncliffe Park
Adelaide,King,Richmond

Dovercourt Village,Dufferin

Scarborough Village
Fairview
,Henry Farm,Oriole

Northwood Park,York University
East Toronto
Harbourfront East
,Toronto Islands,Union Station
Little Portugal,Trinity
East Birchmount Park
,Ionview,Kennedy Park
Bayview Village
CFB Toronto,Downsview East

The Danforth West
,Riverdale
Design E

In [90]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [91]:
toronto_venues.shape

(2223, 7)

In [92]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond\n",100,100,100,100,100,100
Agincourt,5,5,5,5,5,5
"Agincourt North,L'Amoreaux East\n,Milliken,Steeles East\n",3,3,3,3,3,3
"Albion Gardens\n,Beaumond Heights,Humbergate\n,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",10,10,10,10,10,10
"Alderwood,Long Branch",9,9,9,9,9,9
"Bathurst Manor,Downsview North\n,Wilson Heights",16,16,16,16,16,16
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East\n",23,23,23,23,23,23
Berczy Park,55,55,55,55,55,55


In [93]:
toronto_onehot1 = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot1.head(3)
toronto_onehot1.shape

(2223, 275)

In [94]:
del toronto_onehot1['Neighborhood']
toronto_onehot1.head(3)

,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [95]:
toronto_onehot1['Neighborhood'] = toronto_venues['Neighborhood'] 
toronto_onehot1.head(3)

,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Neighborhood
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Parkwoods
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Parkwoods
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Parkwoods


In [96]:
fixed_columns = [toronto_onehot1.columns[-1]] + list(toronto_onehot1.columns[:-1])
toronto_onehot1 = toronto_onehot1[fixed_columns]

toronto_onehot1.head(3)

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [97]:
toronto_grouped = toronto_onehot1.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond\n",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.01,0.0,0.0,0.0,0.0,0.01,0.0,0.01,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0
2,"Agincourt North,L'Amoreaux East\n,Milliken,Ste...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0
3,"Albion Gardens\n,Beaumond Heights,Humbergate\n...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0
4,"Alderwood,Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0


In [98]:
toronto_grouped.shape

(97, 275)

### Top Venues based on facilities in neighborhood

In [99]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond
----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2      Thai Restaurant  0.04
3           Steakhouse  0.04
4  American Restaurant  0.04


----Agincourt----
                venue  freq
0  Chinese Restaurant   0.2
1      Breakfast Spot   0.2
2              Lounge   0.2
3      Sandwich Place   0.2
4        Skating Rink   0.2


----Agincourt North,L'Amoreaux East
,Milliken,Steeles East
----
                       venue  freq
0                       Park  0.67
1                 Playground  0.33
2  Middle Eastern Restaurant  0.00
3                      Motel  0.00
4        Monument / Landmark  0.00


----Albion Gardens
,Beaumond Heights,Humbergate
,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                  venue  freq
0         Grocery Store   0.2
1              Pharmacy   0.1
2   Fried Chicken Joint   0.1
3  Fast Food Restaurant   0.1
4           Pizza Place   0.1


----Alderwood,Long Branch----
   

               venue  freq
0               Park  0.50
1  Convenience Store  0.25
2        Coffee Shop  0.25
3  Accessories Store  0.00
4  Mobile Phone Shop  0.00


----Emery,Humberlea----
                        venue  freq
0              Baseball Field   0.5
1  Construction & Landscaping   0.5
2          Miscellaneous Shop   0.0
3               Movie Theater   0.0
4                       Motel   0.0


----Fairview
,Henry Farm,Oriole
----
                  venue  freq
0        Clothing Store  0.14
1  Fast Food Restaurant  0.08
2           Coffee Shop  0.06
3         Women's Store  0.05
4   Sporting Goods Shop  0.03


----First Canadian Place,Underground city----
                 venue  freq
0          Coffee Shop  0.08
1                 Café  0.08
2                Hotel  0.06
3           Restaurant  0.05
4  American Restaurant  0.04


----Flemingdon Park,Don Mills South
----
                 venue  freq
0           Beer Store  0.10
1          Coffee Shop  0.10
2                  Gym  0

               venue  freq
0         Astrologer   0.2
1        Coffee Shop   0.2
2                Pub   0.2
3      Grocery Store   0.2
4  Accessories Store   0.0


----The Beaches West
,India Bazaar----
              venue  freq
0       Pizza Place  0.08
1              Park  0.08
2    Sandwich Place  0.08
3  Sushi Restaurant  0.04
4               Pub  0.04


----The Danforth West
,Riverdale----
                venue  freq
0    Greek Restaurant  0.24
1         Coffee Shop  0.10
2      Ice Cream Shop  0.07
3           Bookstore  0.05
4  Italian Restaurant  0.05


----The Junction North
,Runnymede
----
               venue  freq
0        Pizza Place  0.25
1           Bus Line  0.25
2  Convenience Store  0.25
3      Grocery Store  0.25
4  Mobile Phone Shop  0.00


----The Kingsway,Montgomery Road
,Old Mill North
----
               venue  freq
0               Park  0.25
1               Pool  0.25
2              River  0.25
3         Smoke Shop  0.25
4  Accessories Store  0.00


----Thorncl

In [102]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [166]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond\n",Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Hotel,Bakery,Asian Restaurant,Clothing Store,Gym
1,Agincourt,Breakfast Spot,Chinese Restaurant,Sandwich Place,Lounge,Skating Rink,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
2,"Agincourt North,L'Amoreaux East\n,Milliken,Ste...",Park,Playground,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
3,"Albion Gardens\n,Beaumond Heights,Humbergate\n...",Grocery Store,Pharmacy,Fast Food Restaurant,Pizza Place,Sandwich Place,Coffee Shop,Beer Store,Liquor Store,Fried Chicken Joint,Donut Shop
4,"Alderwood,Long Branch",Pizza Place,Pool,Pub,Gym,Coffee Shop,Sandwich Place,Pharmacy,Skating Rink,Dim Sum Restaurant,Diner


### Cluster Analysis on the dataset

In [167]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 4, 0, 0, 0, 0, 0, 0, 0])

In [168]:
len(kmeans.labels_)

97

In [169]:
kmeans.labels_

array([0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 3, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4,
       0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 4, 0, 0, 0, 0, 0, 1])

In [170]:
neighborhoods_venues_sorted.shape

(97, 11)

In [171]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = f_toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Fast Food Restaurant,Park,Food & Drink Shop,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Intersection,Pizza Place,Coffee Shop,Hockey Arena,Portuguese Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,0.0,Coffee Shop,Park,Bakery,Café,Pub,Breakfast Spot,Restaurant,Theater,Mexican Restaurant,Italian Restaurant
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,0.0,Clothing Store,Furniture / Home Store,Accessories Store,Boutique,Shoe Store,Miscellaneous Shop,Gift Shop,Event Space,Women's Store,Vietnamese Restaurant
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,0.0,Coffee Shop,Gym,Sushi Restaurant,Japanese Restaurant,Diner,Yoga Studio,Café,Smoothie Shop,Seafood Restaurant,Sculpture Garden


In [172]:
toronto_merged.dropna(inplace=True)

In [173]:
toronto_merged['Cluster Labels'].astype(int, inplace=True)

0      0
1      0
2      0
3      0
4      0
6      0
7      0
8      0
9      0
10     0
11     2
12     1
13     0
14     0
15     0
16     0
17     0
18     0
19     0
20     0
21     0
22     0
23     0
24     0
25     0
26     0
27     0
28     0
29     0
30     0
      ..
70     0
71     0
72     0
73     0
74     0
75     0
76     0
77     0
78     0
79     0
80     0
81     4
82     0
83     4
84     0
85     0
86     0
87     0
88     0
89     4
90     0
91     0
92     0
94     0
95     0
96     0
97     0
98     0
99     3
100    0
Name: Cluster Labels, Length: 97, dtype: int32

In [174]:
toronto_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97 entries, 0 to 100
Data columns (total 16 columns):
Postal Code               97 non-null object
Borough                   97 non-null object
Neighborhood              97 non-null object
Latitude                  97 non-null float64
Longitude                 97 non-null float64
Cluster Labels            97 non-null float64
1st Most Common Venue     97 non-null object
2nd Most Common Venue     97 non-null object
3rd Most Common Venue     97 non-null object
4th Most Common Venue     97 non-null object
5th Most Common Venue     97 non-null object
6th Most Common Venue     97 non-null object
7th Most Common Venue     97 non-null object
8th Most Common Venue     97 non-null object
9th Most Common Venue     97 non-null object
10th Most Common Venue    97 non-null object
dtypes: float64(3), object(13)
memory usage: 12.9+ KB


In [175]:
toronto_merged.head(101)

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Fast Food Restaurant,Park,Food & Drink Shop,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Intersection,Pizza Place,Coffee Shop,Hockey Arena,Portuguese Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,0.0,Coffee Shop,Park,Bakery,Café,Pub,Breakfast Spot,Restaurant,Theater,Mexican Restaurant,Italian Restaurant
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,0.0,Clothing Store,Furniture / Home Store,Accessories Store,Boutique,Shoe Store,Miscellaneous Shop,Gift Shop,Event Space,Women's Store,Vietnamese Restaurant
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,0.0,Coffee Shop,Gym,Sushi Restaurant,Japanese Restaurant,Diner,Yoga Studio,Café,Smoothie Shop,Seafood Restaurant,Sculpture Garden
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,0.0,Fast Food Restaurant,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
7,M3B,North York,Don Mills North\n,43.745906,-79.352188,0.0,Café,Gym / Fitness Center,Caribbean Restaurant,Pool,Japanese Restaurant,Baseball Field,Drugstore,Diner,Discount Store,Dog Run
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937,0.0,Fast Food Restaurant,Pizza Place,Pharmacy,Athletics & Sports,Gastropub,Intersection,Pet Store,Rock Climbing Spot,Bank,Gym / Fitness Center
9,M5B,Downtown Toronto,"Ryerson,Garden District\n",43.657162,-79.378937,0.0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Japanese Restaurant,Theater,Italian Restaurant,Plaza,Diner
10,M6B,North York,Glencairn,43.709577,-79.445073,0.0,Pub,Japanese Restaurant,Playground,Bakery,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore


In [176]:
import matplotlib.cm as cm
import matplotlib.colors as colors


# create map
map_clusters = folium.Map(location=[t_latitude, t_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    #print(cluster)
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster) + str(lat) + str(lon),parse_html=True)
        folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

['#8000ff', '#00b5eb', '#80ffb4', '#ffb360', '#ff0000']


### Analysis produce resuts of the top neighborhood and silimilarities in them which are visualised above.

### Cluster 1

In [177]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0.0,Fast Food Restaurant,Park,Food & Drink Shop,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
1,North York,0.0,Intersection,Pizza Place,Coffee Shop,Hockey Arena,Portuguese Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run
2,Downtown Toronto,0.0,Coffee Shop,Park,Bakery,Café,Pub,Breakfast Spot,Restaurant,Theater,Mexican Restaurant,Italian Restaurant
3,North York,0.0,Clothing Store,Furniture / Home Store,Accessories Store,Boutique,Shoe Store,Miscellaneous Shop,Gift Shop,Event Space,Women's Store,Vietnamese Restaurant
4,Queen's Park,0.0,Coffee Shop,Gym,Sushi Restaurant,Japanese Restaurant,Diner,Yoga Studio,Café,Smoothie Shop,Seafood Restaurant,Sculpture Garden
6,Scarborough,0.0,Fast Food Restaurant,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
7,North York,0.0,Café,Gym / Fitness Center,Caribbean Restaurant,Pool,Japanese Restaurant,Baseball Field,Drugstore,Diner,Discount Store,Dog Run
8,East York,0.0,Fast Food Restaurant,Pizza Place,Pharmacy,Athletics & Sports,Gastropub,Intersection,Pet Store,Rock Climbing Spot,Bank,Gym / Fitness Center
9,Downtown Toronto,0.0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Japanese Restaurant,Theater,Italian Restaurant,Plaza,Diner
10,North York,0.0,Pub,Japanese Restaurant,Playground,Bakery,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore


### Cluster 2

In [178]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,1.0,Bar,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Yoga Studio,Diner
65,North York,1.0,Bar,Bank,Park,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Yoga Studio


### Cluster 3

In [179]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Etobicoke,2.0,Bank,Yoga Studio,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Diner


### Cluster 4

In [180]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
56,North York,3.0,Construction & Landscaping,Baseball Field,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
99,Etobicoke,3.0,Construction & Landscaping,Baseball Field,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant


### Cluster 5

In [181]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,Scarborough,4.0,Playground,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
34,East York,4.0,Park,Coffee Shop,Convenience Store,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
63,York,4.0,Park,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
81,Central Toronto,4.0,Summer Camp,Playground,Trail,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
83,Scarborough,4.0,Park,Playground,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
89,Downtown Toronto,4.0,Park,Trail,Playground,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
